In [40]:
import torch

import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/") # PATH TO PYSEQM

import seqm
from seqm.seqm_functions.constants import Constants
from seqm.seqm_functions.make_dm_guess import make_dm_guess
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz

from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import os
import numpy as np

seqm.seqm_functions.scf_loop.debug=True
seqm.seqm_functions.scf_loop.MAX_ITER=500

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [41]:
species = torch.as_tensor([[16,16],[22,22],[22,16],[35,17],[24,22]],dtype=torch.int64, device=device)

coordinates = torch.tensor([
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
                 ], device=device)


# species = torch.as_tensor([[17,17,17,17,6]],dtype=torch.int64, device=device)

# coordinates = torch.tensor([
#                   [
#                    [0.0000,    0.0000,    1.7700],
#                    [1.7700,    0.0000,    0.0000],
#                    [-1.250,    1.2500,    0.0000],
#                    [-1.250,   -1.2500,    0.0000],
#                    [0.0000,    0.0000,    0.0000]
#                   ]
#                  ], device=device)

const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : False,
                    "Hf_flag": True,
                    'scf_backward' : 0,
                    'UHF' : False,
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)
esdriver = Electronic_Structure(seqm_parameters).to(device)
esdriver(molecules)

scf  0 162.9290240440854 5 0.20192980766296387
scf  1 46.45832159787335 5 0.140899658203125
scf  2 8.993806268891092 5 0.12546849250793457
scf  3 1.9401746586516424 5 0.0876169204711914
scf  4 0.6026012521201665 5 0.08928966522216797
scf  5 0.20288523476443743 5 0.12214279174804688
scf  6 0.10973329959142575 5 0.13244390487670898
scf  7 0.10972942315572709 5 0.15432000160217285
scf  8 0.11960946471526768 5 0.15456676483154297
scf  9 0.13918681369563046 5 0.15081357955932617
scf  10 0.17202618774055622 4 0.13640499114990234
scf  11 0.22501517146642414 4 0.11614847183227539
scf  12 0.3082230790109861 3 0.11536908149719238
scf  13 0.43010396622415215 3 0.09498786926269531
scf  14 0.5795471243194754 2 0.08931231498718262
scf  15 0.7015496985500818 2 0.06026005744934082
scf  16 0.7211362889869122 2 0.07138586044311523
scf  17 0.6237496539345386 1 0.12117838859558105
scf  18 0.46699327979746386 1 0.06827139854431152
scf  19 0.31369072032782697 1 0.05253195762634277
scf  20 0.1945768335449429

# Total E

In [42]:
molecules.Etot[0] # S2. MOPAC Etot = -333.8287 EV

tensor(-331.8060, device='cuda:0')

In [43]:
molecules.Etot[1] # Ti2. MOPAC Etot = -120.0651 EV

tensor(-120.0784, device='cuda:0')

In [44]:
molecules.Etot[2] # TiS. MOPAC Etot = -221.2701 EV  !!!!!!!!!!!!!!!!!!!!!

tensor(-283.9792, device='cuda:0')

In [45]:
molecules.Etot[3] # BrCl. MOPAC Etot = -458.5855 EV

tensor(-458.4182, device='cuda:0')

In [46]:
molecules.Etot[4] # CrTi. MOPAC Etot = -150.3163 EV  !!!!!!!!!!!!!!!!!!!!!

tensor(-366.0346, device='cuda:0')

# Nuclear repulsion

In [47]:
molecules.Enuc[0] # S2. MOPAC Enuc = 290.8693 EV

tensor(290.8642, device='cuda:0')

In [48]:
molecules.Enuc[1] # Ti2. MOPAC Enuc = 97.3923 EV

tensor(97.3926, device='cuda:0')

In [49]:
molecules.Enuc[2] # TiS. MOPAC Enuc = 176.1518 EV

tensor(176.1500, device='cuda:0')

In [50]:
molecules.Enuc[3] # BrCl. MOPAC Enuc = 391.1161 EV

tensor(391.1114, device='cuda:0')

In [51]:
molecules.Enuc[4] # CrTi. MOPAC Enuc = 251.0941 EV  !!!!!!!!!!!!!!!!!!!!!

tensor(145.6023, device='cuda:0')